In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def scrape_subreddit(subreddit_name, limit=25):
    headers = {'User-Agent': 'Mozilla/5.0'}
    url = f"https://old.reddit.com/r/{subreddit_name}/"

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch data from r/{subreddit_name}, status code: {response.status_code}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')
    posts_data = []

    for post in soup.find_all('div', class_='thing')[:limit]:
        try:
            title = post.find('a', class_='title').text.strip()
            
            upvotes_tag = post.find('div', class_='score unvoted')
            if upvotes_tag:
                upvote_text = upvotes_tag.get('title')
                upvotes = int(upvote_text.replace('k', '000')) if upvote_text and upvote_text.isdigit() else 0
            else:
                upvotes = 0

            time_posted = post.find('time')['title'] if post.find('time') else "Unknown"
            link = "https://www.reddit.com" + post.get('data-permalink')

            posts_data.append({
                'Title': title,
                'Upvotes': upvotes,
                'Posted On': time_posted,
                'Post URL': link
            })
        except Exception as e:
            print("Skipping a post due to error:", e)

    df = pd.DataFrame(posts_data)
    return df


In [3]:
df = scrape_subreddit("Python")

In [4]:
df.head()

,Title,Upvotes,Posted On,Post URL
0,PyCon US 2025 is next week!,11,Tue May 6 14:30:48 2025 UTC,https://www.reddit.com/r/Python/comments/1kg5x...
1,"Thursday Daily Thread: Python Careers, Courses...",3,Thu May 15 00:00:31 2025 UTC,https://www.reddit.com/r/Python/comments/1kmuf...
2,Microsoft layoffs hit Faster CPython team - in...,435,Thu May 15 01:39:39 2025 UTC,https://www.reddit.com/r/Python/comments/1kmwd...
3,Look for people who can promote my products fo...,1,Unknown,https://www.reddit.com/user/Great_Advice_574/c...
4,Blame as a Service: Open-source for Blaming Ot...,37,Thu May 15 02:27:28 2025 UTC,https://www.reddit.com/r/Python/comments/1kmxa...
